## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-16-14-53-13 +0000,nyt,Live Updates: Israeli Troops Push Into Gaza Ci...,https://www.nytimes.com/live/2025/09/16/world/...
1,2025-09-16-14-51-27 +0000,bbc,Malawians await presidential poll result in vo...,https://www.bbc.com/news/articles/c5y56req0g1o...
2,2025-09-16-14-47-53 +0000,nyt,Trump Signs Off on Sending National Guard to M...,https://www.nytimes.com/2025/09/15/us/politics...
3,2025-09-16-14-44-00 +0000,wsj,FBI Director Kash Patel is defending his work ...,https://www.wsj.com/politics/policy/fbi-direct...
4,2025-09-16-14-43-35 +0000,nyt,Federal Reserve to Meet Under Cloud of Trump’s...,https://www.nytimes.com/2025/09/16/business/fe...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2299/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
19,trump,57
35,kirk,34
34,charlie,30
107,new,20
5,gaza,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
18,2025-09-16-14-28-43 +0000,wapo,President Donald Trump praised FBI Director Ka...,https://www.washingtonpost.com,166
3,2025-09-16-14-44-00 +0000,wsj,FBI Director Kash Patel is defending his work ...,https://www.wsj.com/politics/policy/fbi-direct...,134
203,2025-09-16-00-41-00 +0000,wsj,President Trump got a win when the Senate conf...,https://www.wsj.com/economy/central-banking/st...,127
17,2025-09-16-14-28-43 +0000,wapo,Live updates: FBI director faces questions on ...,https://www.washingtonpost.com/nation/2025/09/...,120
204,2025-09-16-00-39-31 +0000,nypost,Senate confirms Trump Fed pick Stephen Miran a...,https://nypost.com/2025/09/15/us-news/senate-c...,119


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
18,166,2025-09-16-14-28-43 +0000,wapo,President Donald Trump praised FBI Director Ka...,https://www.washingtonpost.com
15,83,2025-09-16-14-32-00 +0000,wsj,Israel Launches New Ground Offensive Seeking t...,https://www.wsj.com/world/middle-east/israel-l...
56,62,2025-09-16-13-19-00 +0000,wsj,A federal appeals court on Monday night reject...,https://www.wsj.com/economy/central-banking/ap...
203,61,2025-09-16-00-41-00 +0000,wsj,President Trump got a win when the Senate conf...,https://www.wsj.com/economy/central-banking/st...
14,55,2025-09-16-14-32-44 +0000,startribune,New York judge tosses terrorism charges agains...,https://www.startribune.com/luigi-mangione-due...
170,51,2025-09-16-04-22-08 +0000,nypost,Old man who falsely claimed he shot Charlie Ki...,https://nypost.com/2025/09/16/us-news/old-man-...
259,39,2025-09-15-20-29-55 +0000,cbc,"Carney, Poilievre come face-to-face for the fi...",https://www.cbc.ca/news/politics/carney-poilie...
233,38,2025-09-15-22-18-14 +0000,nypost,"US, China near TikTok deal to finally resolve ...",https://nypost.com/2025/09/15/business/us-chin...
0,35,2025-09-16-14-53-13 +0000,nyt,Live Updates: Israeli Troops Push Into Gaza Ci...,https://www.nytimes.com/live/2025/09/16/world/...
254,35,2025-09-15-20-50-24 +0000,cbc,Trump says U.S. has struck another alleged dru...,https://www.cbc.ca/news/world/us-drug-boat-ven...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
